# Medical Data Processing and Document Analysis
This notebook processes company FAQs and medical data from Mayo Clinic for document analysis.

## 1. Environment Setup and Dependencies

In [1]:

# =============================================================================
# ENVIRONMENT SETUP AND DEPENDENCIES
# =============================================================================

# Standard Library Imports
import os
import sys
import json
import uuid
import time
import logging
import warnings
from pathlib import Path
from datetime import datetime
from typing import List, Dict, Any, Tuple, Optional
import pickle


# Third-party Library Imports
import pandas as pd
import pytz
from dotenv import load_dotenv
from huggingface_hub import login
from IPython.display import display, Markdown

# LangChain Core Imports
from langchain.schema import Document
from langchain.tools import tool
from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferWindowMemory
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.retrievers import EnsembleRetriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage

# LangChain Text Processing
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    MarkdownHeaderTextSplitter,
)

# LangChain Embeddings and Vector Stores
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever

# LangChain Tools
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools import TavilySearchResults

# LangChain Models
from langchain_openai import ChatOpenAI

# Pydantic for Data Validation
from pydantic.v1 import BaseModel, Field

# =============================================================================
# CONFIGURATION AND SETUP
# =============================================================================

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Utility function for Markdown display
def print_markdown(text: str) -> None:
    """Display text as formatted Markdown in Jupyter notebook."""
    display(Markdown(text))

# =============================================================================
# ENVIRONMENT INITIALIZATION
# =============================================================================

# Load environment variables
load_dotenv()

# Authenticate with Hugging Face
huggingface_token = os.getenv("HUGGINGFACE_HUB_TOKEN")
if huggingface_token:
    login(huggingface_token)
    logger.info("Successfully authenticated with Hugging Face")
else:
    logger.warning("HUGGINGFACE_HUB_TOKEN not found in environment variables")

# Verify OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    logger.error("OPENAI_API_KEY not found in environment variables")
    raise ValueError("OpenAI API key is required")

# Verify Tavily API key for web search
tavily_api_key = os.getenv("TAVILY_API_KEY")
if not tavily_api_key:
    logger.warning("TAVILY_API_KEY not found - web search functionality will be limited")

# =============================================================================
# DIRECTORY CONFIGURATION
# =============================================================================

# Configure data directories
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Added to Python path: {project_root}")

# Now you can import from src
from src.config import DATA_DIR, COMPANY_INFO_DIR, VECTOR_STORE_DIR

# Verify required data files exist
required_files = [
    COMPANY_INFO_DIR / Path('info.md'),
    COMPANY_INFO_DIR / Path('FAQ.csv')
]

for file_path in required_files:
    if file_path.exists():
        logger.info(f"Found required file: {file_path}")
    else:
        logger.warning(f"Required file not found: {file_path}")

logger.info("Environment setup completed successfully!")

2025-08-07 21:28:19,270 - __main__ - INFO - Successfully authenticated with Hugging Face


Added to Python path: d:\Work\Agentic-Medical-RAG-Chatbot


2025-08-07 21:28:19,578 - src.config - INFO - Successfully logged into HuggingFace
2025-08-07 21:28:25,449 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-small
2025-08-07 21:28:33,191 - src.config - INFO - Configuration validation completed
2025-08-07 21:28:33,193 - __main__ - INFO - Found required file: d:\Work\Agentic-Medical-RAG-Chatbot\data\raw_company_info\info.md
2025-08-07 21:28:33,193 - __main__ - INFO - Found required file: d:\Work\Agentic-Medical-RAG-Chatbot\data\raw_company_info\FAQ.csv
2025-08-07 21:28:33,194 - __main__ - INFO - Environment setup completed successfully!


## 2. Data Loading and Processing
### 2.1 Company FAQ Processing

In [2]:
def load_faq_documents(faq_path: Path) -> List[Document]:
    """
    Load and process FAQ documents from CSV file.
    
    Args:
        faq_path: Path to the FAQ CSV file
        
    Returns:
        List of Document objects
    """
    try:
        # Validate file exists
        if not faq_path.exists():
            raise FileNotFoundError(f"FAQ file not found at {faq_path}")
            
        df = pd.read_csv(faq_path)
        
        # Validate required columns
        required_cols = ['Question', 'Answer']
        if not all(col in df.columns for col in required_cols):
            raise ValueError(f"CSV must contain columns: {required_cols}")
            
        documents = []
        for idx, row in df.iterrows():
            content = f"Question: {row.get('Question', '')}\nAnswer: {row.get('Answer', '')}"
            
            doc = Document(
                page_content=content,
                metadata={
                    "source": "company_faq",
                    "type": "faq", 
                    "doc_id": f"{idx}", 
                }
            )
            documents.append(doc)
            
        logger.info(f"Loaded {len(documents)} FAQ documents")
        return documents
        
    except Exception as e:
        logger.error(f"Error loading FAQ documents: {str(e)}")
        raise


### 2.2 Company Information File Processing

In [3]:
def load_company_info(info_path: Path) -> Document:
    """
    Load company information from markdown file.
    
    Args:
        info_path: Path to the company info markdown file
        
    Returns:
        Document object containing company info
    """
    try:
        # Validate file exists
        if not info_path.exists():
            raise FileNotFoundError(f"Info file not found at {info_path}")
            
        with open(info_path, 'r', encoding='utf-8') as f:
            content = f.read()
            
        doc = Document(
            page_content=content,
            metadata={
                "source": "company_info",
                "type": "general_info",
                "filename": info_path.name,
                "doc_id": "company_info_main"
            }
        )
        logger.info("Loaded company info document")
        return doc
        
    except Exception as e:
        logger.error(f"Error loading company info: {str(e)}")
        raise


In [4]:
print(COMPANY_INFO_DIR / Path('FAQ.csv'))
# Load all documents
faq_path = COMPANY_INFO_DIR / Path('FAQ.csv')
info_path = COMPANY_INFO_DIR / Path('info.md')

company_documents = load_faq_documents(faq_path)
company_documents.append(load_company_info(info_path))

print(f"✅ Loaded {len(company_documents)} company documents")
company_documents[-5:]

2025-08-07 21:28:45,961 - __main__ - INFO - Loaded 262 FAQ documents
2025-08-07 21:28:45,962 - __main__ - INFO - Loaded company info document


d:\Work\Agentic-Medical-RAG-Chatbot\data\raw_company_info\FAQ.csv
✅ Loaded 263 company documents


[Document(metadata={'source': 'company_faq', 'type': 'faq', 'doc_id': '258'}, page_content='Question: هل يمكنني إلغاء موعد الصيدلية والتوصيل؟\nAnswer: نعم، يمكن إلغاء الطلب قبل عملية الشحن، يرجى التواصل مع الدعم الفني في أقرب وقت ممكن.'),
 Document(metadata={'source': 'company_faq', 'type': 'faq', 'doc_id': '259'}, page_content='Question: هل هناك رسوم خفية على الخدمات؟\nAnswer: لا، جميع الرسوم والتكاليف موضحة بشفافية تامة قبل إتمام عملية الدفع.'),
 Document(metadata={'source': 'company_faq', 'type': 'faq', 'doc_id': '260'}, page_content='Question: هل يمكنني الاحتفاظ بسجلي الطبي على جهازي الخاص؟\nAnswer: يمكنك تنزيل نسخة من ملخص الاستشارات والتقارير المتاحة لك على المنصة.'),
 Document(metadata={'source': 'company_faq', 'type': 'faq', 'doc_id': '261'}, page_content='Question: هل تتوافق المنصة مع متصفحات الإنترنت المختلفة؟\nAnswer: نعم، تم تصميم المنصة لتكون متوافقة مع معظم متصفحات الإنترنت الشائعة (كروم، فايرفوكس، سفاري، إيدج).'),
 Document(metadata={'source': 'company_info', 'type': 'ge

## 3. Text Splitting Configuration and Processing

In [ ]:
recursive_500 = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)


markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "company_title"),
        ("##", "section"),
    ]
)

# Process company documents
company_chunks = []

for doc in company_documents:
    if doc.metadata.get("type") == "general_info":
        # Use markdown splitter for info.md
        split_docs = markdown_splitter.split_text(doc.page_content)
        for d in split_docs:
            d.metadata.update(doc.metadata)
        company_chunks.extend(split_docs)
    else:
        # Use recursive splitter for FAQs
        split_docs = recursive_500.split_documents([doc])
        company_chunks.extend(split_docs)

print(f"Company chunks: {len(company_chunks)}")

# with open("company_chunks.pkl", "wb") as f:
#   pickle.dump(company_chunks, f)

Company chunks: 274


### 3.1 Evaluation of Document Chunk Quality

In [6]:
for i, doc in enumerate(company_chunks[-25:]):
    print(f"\n--- Chunk {i+1} ---")
    print("Metadata:", doc.metadata)
    print("content:\n",doc.page_content)


--- Chunk 1 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '249'}
content:
 Question: هل هناك حدود لعدد الاستشارات التي يمكنني الحصول عليها؟
Answer: يعتمد ذلك على الباقة التي اشتركت بها. بعض الباقات توفر عددًا محدودًا، بينما تتيح أخرى استشارات غير محدودة.

--- Chunk 2 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '250'}
content:
 Question: هل يمكنني الحصول على رأي طبي ثانٍ؟
Answer: نعم، يمكنك حجز استشارة مع طبيب آخر للحصول على رأي طبي ثانٍ، وذلك ضمن الخدمات المتاحة.

--- Chunk 3 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '251'}
content:
 Question: هل تتوفر خدمات طب الأسنان عن بعد؟
Answer: حاليًا لا نقدم خدمات طب الأسنان عن بعد، ولكننا نسعى لتوسيع نطاق التخصصات مستقبلاً.

--- Chunk 4 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '252'}
content:
 Question: هل يمكنني استشارة طبيب أخصائي دون تحويل من طبيب عام؟
Answer: نعم، يمكنك حجز موعد مباشر مع الطبيب الأخصائي الذي تختاره.

--- Chunk 5 ---
Metadata: {'source':

## 4. Testing Embedding Models & Retrieval Types

### 4.1 Define Retrieval Functions

In [7]:
# Create vector retriever
def create_vector_retriever(documents: List[Document], embedding_model: str, k: int = 5):
   """
   Create a vector-based retriever.
   
   Args:
       documents: List of documents to index
       embedding_model: Name of the embedding model
       k: Number of documents to retrieve
   
   Returns:
       Tuple of (vector_retriever, vector_store)
   """
   embeddings = HuggingFaceEmbeddings(
       model_name=embedding_model,
       model_kwargs={'device': 'cpu'},
       encode_kwargs={'normalize_embeddings': True}
   )
   
   # Create vector store
   vector_store = FAISS.from_documents(documents, embeddings)
   vector_retriever = vector_store.as_retriever(search_kwargs={"k": k})
   
   return vector_retriever, vector_store

# Create BM25 retriever
def create_bm25_retriever(documents: List[Document], k: int = 5):
   """
   Create a BM25-based retriever.
   
   Args:
       documents: List of documents to index
       k: Number of documents to retrieve
   
   Returns:
       BM25Retriever object
   """
   bm25_retriever = BM25Retriever.from_documents(documents)
   bm25_retriever.k = k
   
   return bm25_retriever

# Create ensemble (hybrid) retriever
def create_hybrid_retriever(bm25_retriever, vector_retriever, weights: List[float] = [0.5, 0.5]):
   """
   Create a hybrid retriever combining BM25 and vector search.
   
   Args:
       bm25_retriever: BM25 retriever object
       vector_retriever: Vector retriever object
       weights: List of weights for [BM25, Vector] retrievers
   
   Returns:
       EnsembleRetriever object
   """
   ensemble_retriever = EnsembleRetriever(
       retrievers=[bm25_retriever, vector_retriever],
       weights=weights
   )

   return ensemble_retriever

print("🔧 Retrieval functions defined!")

🔧 Retrieval functions defined!


### 4.2 Evaluation for All Models and Retriever Types

In [8]:
def evaluate_retriever(retriever, queries: List[str], retriever_type: str) -> Dict[str, Any]:
    """
    Evaluate a retriever on given queries.

    Args:
        retriever: The retriever to evaluate
        queries: List of queries to test
        retriever_type: Type of retriever ('vector' or 'hybrid')

    Returns:
        Dict containing evaluation results
    """
    results = []
    total_time = 0

    for i, query in enumerate(queries):
        start_time = time.time()

        # Retrieve documents
        retrieved_docs = retriever.get_relevant_documents(query)

        end_time = time.time()
        query_time = end_time - start_time
        total_time += query_time

        # Store results
        query_result = {
            'query_id': i + 1,
            'query': query,
            'retriever_type': retriever_type,
            'num_retrieved': len(retrieved_docs),
            'retrieval_time': query_time,
            'retrieved_docs': [
                {
                    'content': doc.page_content[:500] + "..." if len(doc.page_content) > 500 else doc.page_content,  
                    'metadata': doc.metadata,
                    'relevance_score': getattr(doc, 'metadata', {}).get('score', 'N/A')
                }
                for doc in retrieved_docs[:10]
            ]
        }
        results.append(query_result)

    return {
        'results': results,
        'total_time': total_time,
        'avg_time_per_query': total_time / len(queries) if queries else 0
    }


# Embedding models to test
embedding_models = [
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "intfloat/multilingual-e5-small",
    "ibm-granite/granite-embedding-107m-multilingual"
]

# Test queries 
test_queries = [
    "ما هي الخدمات الرئيسية التي تقدمونها",      # What are your main services?
    "كيف يمكنني التواصل معكم",                   # How can I contact you?
    "ما هي أوقات العمل",                        # What are your working hours?
]

k = 5


print("🎯 Configuration loaded:")
print(f"📊 Embedding models: {len(embedding_models)}")
print(f"🏢 Company queries: {len(test_queries)}")
print(f"Using k = {k} 'Number of retrieved documents':")

# Run Evaluation for All Models and Retriever Types
evaluation_results = {}

print("🚀 Starting evaluation of embedding models with different retriever types...\n")

for model_name in embedding_models:
    print(f"📊 Evaluating: {model_name}")
    print("-" * 80)

    model_start_time = time.time()

    try:
        # Create vector retriever
        print("🔍 Creating vector retriever...")
        vector_retriever, vector_store = create_vector_retriever(
            company_chunks, model_name, k=k
        )
        
        # Create BM25 retriever
        print("📝 Creating BM25 retriever...")
        bm25_retriever = create_bm25_retriever(company_chunks, k=k)
        
        # Create hybrid retriever
        print("🔄 Creating hybrid retriever...")
        hybrid_retriever = create_hybrid_retriever(
            bm25_retriever, vector_retriever, weights=[0.3, 0.7]
        )

        # Evaluate vector-only retriever
        print("🔍 Evaluating vector retriever...")
        vector_results = evaluate_retriever(vector_retriever, test_queries, "vector")

        # Evaluate hybrid retriever
        print("🔄 Evaluating hybrid retriever...")
        hybrid_results = evaluate_retriever(hybrid_retriever, test_queries, "hybrid")

        model_end_time = time.time()
        total_model_time = model_end_time - model_start_time

        # Store results
        evaluation_results[model_name] = {
            'vector_results': vector_results,
            'hybrid_results': hybrid_results,
            'total_time': total_model_time,
            'vector_avg_time': vector_results['avg_time_per_query'],
            'hybrid_avg_time': hybrid_results['avg_time_per_query']
        }

        print(f"✅ Completed in {total_model_time:.2f} seconds")
        print(f"🔍 Vector avg: {vector_results['avg_time_per_query']:.3f}s per query")
        print(f"🔄 Hybrid avg: {hybrid_results['avg_time_per_query']:.3f}s per query")

    except Exception as e:
        print(f"❌ Error with {model_name}: {str(e)}")
        evaluation_results[model_name] = {'error': str(e)}

    print("\n")

print("🎉 Evaluation completed for all models and retriever types!")

# Cell 6: Display Detailed Results
def display_results():
    """Display detailed results in a professional format."""

    print("=" * 100)
    print("📊 EMBEDDING MODELS & RETRIEVER TYPES EVALUATION RESULTS")
    print("=" * 100)

    # Summary table
    summary_data = []

    for model_name, results in evaluation_results.items():
        if 'error' not in results:
            summary_data.append({
                'Model': model_name.split('/')[-1],  # Short name
                'Total Time (s)': f"{results['total_time']:.2f}",
                'Vector Avg (s)': f"{results['vector_avg_time']:.3f}",
                'Hybrid Avg (s)': f"{results['hybrid_avg_time']:.3f}",
                'Queries Tested': len(results['vector_results']['results'])
            })

    summary_df = pd.DataFrame(summary_data)
    print("\n📈 PERFORMANCE SUMMARY")
    print("-" * 70)
    print(summary_df.to_string(index=False))

    # Detailed results for each model
    for model_name, results in evaluation_results.items():
        if 'error' not in results:
            print(f"\n🔍 DETAILED RESULTS: {model_name}")
            print("-" * 80)

            # Vector retriever results
            print("\n🔍 VECTOR RETRIEVER RESULTS:")
            for result in results['vector_results']['results']:
                print(f"\nQuery {result['query_id']}: {result['query']}")
                print(f"⏱️  Retrieval time: {result['retrieval_time']:.3f}s")
                print(f"📄 Retrieved {result['num_retrieved']} documents:")

                for i, doc in enumerate(result['retrieved_docs'], 1):  
                    print(f"  {i}. {doc['content']}")
                    print(f"     📁 Source: {doc['metadata'].get('source', 'N/A')}")

            # Hybrid retriever results
            print(f"\n🔄 HYBRID RETRIEVER RESULTS:")
            for result in results['hybrid_results']['results']:
                print(f"\nQuery {result['query_id']}: {result['query']}")
                print(f"⏱️  Retrieval time: {result['retrieval_time']:.3f}s")
                print(f"📄 Retrieved {result['num_retrieved']} documents:")

                for i, doc in enumerate(result['retrieved_docs'], 1):  
                    print(f"  {i}. {doc['content']}")
                    print(f"     📁 Source: {doc['metadata'].get('source', 'N/A')}")

# Display results
display_results()


2025-08-07 21:29:55,671 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-mpnet-base-v2


🎯 Configuration loaded:
📊 Embedding models: 3
🏢 Company queries: 3
Using k = 5 'Number of retrieved documents':
🚀 Starting evaluation of embedding models with different retriever types...

📊 Evaluating: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
--------------------------------------------------------------------------------
🔍 Creating vector retriever...


2025-08-07 21:30:06,767 - faiss.loader - INFO - Loading faiss with AVX512 support.
2025-08-07 21:30:06,768 - faiss.loader - INFO - Could not load library with AVX512 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx512'")
2025-08-07 21:30:06,768 - faiss.loader - INFO - Loading faiss with AVX2 support.
2025-08-07 21:30:06,796 - faiss.loader - INFO - Successfully loaded faiss with AVX2 support.
2025-08-07 21:30:06,802 - faiss - INFO - Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.
2025-08-07 21:30:07,005 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-small


📝 Creating BM25 retriever...
🔄 Creating hybrid retriever...
🔍 Evaluating vector retriever...
🔄 Evaluating hybrid retriever...
✅ Completed in 11.33 seconds
🔍 Vector avg: 0.033s per query
🔄 Hybrid avg: 0.030s per query


📊 Evaluating: intfloat/multilingual-e5-small
--------------------------------------------------------------------------------
🔍 Creating vector retriever...


2025-08-07 21:30:17,322 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: ibm-granite/granite-embedding-107m-multilingual


📝 Creating BM25 retriever...
🔄 Creating hybrid retriever...
🔍 Evaluating vector retriever...
🔄 Evaluating hybrid retriever...
✅ Completed in 10.32 seconds
🔍 Vector avg: 0.013s per query
🔄 Hybrid avg: 0.012s per query


📊 Evaluating: ibm-granite/granite-embedding-107m-multilingual
--------------------------------------------------------------------------------
🔍 Creating vector retriever...
📝 Creating BM25 retriever...
🔄 Creating hybrid retriever...
🔍 Evaluating vector retriever...
🔄 Evaluating hybrid retriever...
✅ Completed in 8.29 seconds
🔍 Vector avg: 0.007s per query
🔄 Hybrid avg: 0.007s per query


🎉 Evaluation completed for all models and retriever types!
📊 EMBEDDING MODELS & RETRIEVER TYPES EVALUATION RESULTS

📈 PERFORMANCE SUMMARY
----------------------------------------------------------------------
                                Model Total Time (s) Vector Avg (s) Hybrid Avg (s)  Queries Tested
paraphrase-multilingual-mpnet-base-v2          11.33          0.033          0.03

### 4.3 LLM Evaluation 

In [9]:
def prepare_llm_evaluation_prompt():
    """Prepare comprehensive prompt for LLM evaluation with retrieved documents analysis."""

    prompt = """
# Embedding Models and Retriever Types Evaluation Report for Company RAG System

I have tested 3 multilingual embedding models with 2 different retriever types for a company information retrieval system. Please analyze the results including the retrieved documents' relevance and recommend the best embedding model and the best retriever type.

## Tested Embedding Models:
1. sentence-transformers/paraphrase-multilingual-mpnet-base-v2
2. intfloat/multilingual-e5-small
3. ibm-granite/granite-embedding-107m-multilingual

## Tested Retriever Types:
1. **Vector Retriever:** Uses vector search only
2. **Hybrid Retriever:** Combines BM25 and vector search (Weights: 30% BM25, 70% Vector)

## Test Queries with Retrieved Documents Analysis:
"""

    for i, query in enumerate(test_queries, 1):
        prompt += f"\n### Query {i}: \"{query}\"\n"
        
        # Add retrieved documents for each model and retriever type
        for model_name, results in evaluation_results.items():
            if 'error' not in results:
                prompt += f"\n**{model_name}:**\n"
                
                # Vector retriever results for this query
                vector_result = results['vector_results']['results'][i-1]
                prompt += f"\n*Vector Retriever (Time: {vector_result['retrieval_time']:.3f}s):*\n"
                for j, doc in enumerate(vector_result['retrieved_docs'], 1):
                    prompt += f"{j}. \"{doc['content'][:300]}{'...' if len(doc['content']) > 300 else ''}\"\n"
                    prompt += f"   Source: {doc['metadata'].get('source', 'N/A')}\n"
                
                # Hybrid retriever results for this query
                hybrid_result = results['hybrid_results']['results'][i-1]
                prompt += f"\n*Hybrid Retriever (Time: {hybrid_result['retrieval_time']:.3f}s):*\n"
                for j, doc in enumerate(hybrid_result['retrieved_docs'], 1):
                    prompt += f"{j}. \"{doc['content'][:300]}{'...' if len(doc['content']) > 300 else ''}\"\n"
                    prompt += f"   Source: {doc['metadata'].get('source', 'N/A')}\n"

    prompt += "\n## Performance Summary:\n\n"

    # Add performance data summary
    for model_name, results in evaluation_results.items():
        if 'error' not in results:
            prompt += f"""
### {model_name}
- **Total Processing Time:** {results['total_time']:.2f} seconds
- **Vector Retriever Average Time:** {results['vector_avg_time']:.3f} seconds per query
- **Hybrid Retriever Average Time:** {results['hybrid_avg_time']:.3f} seconds per query
"""

    prompt += """

## Evaluation Criteria:
1. **Performance Speed:** Faster retrieval times
2. **Document Relevance:** How well retrieved documents match the query intent
3. **Multilingual Support:** Better handling of Arabic queries
4. **Result Quality:** Accuracy and relevance of retrieved documents
5. **Consistency:** Stable performance and relevance across different queries

## What I Need From You:
1. **Rank the Embedding Models:** (1st, 2nd, 3rd) with justification based on both speed AND document relevance
2. **Compare Retriever Types:** Which is better - Vector or Hybrid? Consider both speed and relevance
3. **Document Relevance Analysis:** For each query, which model+retriever combination returned the most relevant documents?
4. **Final Recommendation:** Best combination (embedding model + retriever type) considering all factors
5. **Detailed Analysis:** Strengths and weaknesses of each combination in terms of:
   - Speed performance
   - Document relevance quality
   - Consistency across queries
6. **Production Considerations:** Recommendations for production environment usage

## Objective:
Select the best combination that provides the optimal balance between:
- Response speed
- Quality and relevance of retrieved results
- Efficiency in processing Arabic text
- Performance stability
- Document-query matching accuracy

## Additional Context:
- The system will be used for company information retrieval
- Arabic language support is crucial
- Both speed and accuracy are important
- The system should return highly relevant documents
- Consider scalability for production use

## Analysis Instructions:
Please analyze each query's results and evaluate:
1. Which retrieved documents are most relevant to each query
2. Which model consistently returns better relevant documents
3. Whether vector or hybrid retrieval provides better document relevance
4. The trade-off between speed and relevance quality

Please structure your response with clear sections for:
1. Document Relevance Analysis by Query
2. Speed Performance Analysis
3. Overall Model Ranking
4. Retriever Type Comparison
5. Final Recommendation with Justification
"""

    return prompt

# Prepare the evaluation prompt
llm_prompt = prepare_llm_evaluation_prompt()
print("📝 LLM evaluation prompt prepared!")
print(f"📏 Prompt length: {len(llm_prompt)} characters")
display(Markdown(llm_prompt))

📝 LLM evaluation prompt prepared!
📏 Prompt length: 27846 characters



# Embedding Models and Retriever Types Evaluation Report for Company RAG System

I have tested 3 multilingual embedding models with 2 different retriever types for a company information retrieval system. Please analyze the results including the retrieved documents' relevance and recommend the best embedding model and the best retriever type.

## Tested Embedding Models:
1. sentence-transformers/paraphrase-multilingual-mpnet-base-v2
2. intfloat/multilingual-e5-small
3. ibm-granite/granite-embedding-107m-multilingual

## Tested Retriever Types:
1. **Vector Retriever:** Uses vector search only
2. **Hybrid Retriever:** Combines BM25 and vector search (Weights: 30% BM25, 70% Vector)

## Test Queries with Retrieved Documents Analysis:

### Query 1: "ما هي الخدمات الرئيسية التي تقدمونها"

**sentence-transformers/paraphrase-multilingual-mpnet-base-v2:**

*Vector Retriever (Time: 0.042s):*
1. "Question: ما هي الخدمات التي تقدمها الشركة؟
Answer: تشمل الخدمات الاستشارات الطبية عبر الفيديو، الكشف الجسدي في العيادة، مراجعة التقارير الطبية والتحاليل، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، وخدمات الرعاية النفسية والدعم الصحي. كما تقدم الشركة تحليل نتائج المختبر، نظام السجل..."
   Source: company_faq
2. "Question: ما هي شروط الاشتراك في الخدمات؟
Answer: باستخدامك لخدمات الشركة، فإنك توافق على الالتزام بالشروط والأحكام."
   Source: company_faq
3. "Question: ما هي الشروط التي تحكم استخدام خدمات الشركة؟
Answer: من الشروط الأساسية عدم استخدام الخدمة في حالات الطوارئ، وتقديم معلومات صحيحة وكاملة ودقيقة، وأن الشركة غير مسؤولة عن أي سوء استخدام للخدمة."
   Source: company_faq
4. "Question: ما هي أنواع الباقات والعضويات التي تقدمونها؟
Answer: تقدم الشركة خيارات اشتراك شهرية أو سنوية توفر خصومات إضافية ووصولاً غير محدود لبعض الخدمات."
   Source: company_faq
5. "Question: هل تقدمون خدمات للشركات؟
Answer: نعم، نوفر حزم رعاية صحية رقمية للشركات والمؤسسات لرعاية موظفيهم، وبرامج صحية مخصصة لتعزيز رفاهية الموظفين."
   Source: company_faq

*Hybrid Retriever (Time: 0.029s):*
1. "Question: ما هي الخدمات التي تقدمها الشركة؟
Answer: تشمل الخدمات الاستشارات الطبية عبر الفيديو، الكشف الجسدي في العيادة، مراجعة التقارير الطبية والتحاليل، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، وخدمات الرعاية النفسية والدعم الصحي. كما تقدم الشركة تحليل نتائج المختبر، نظام السجل..."
   Source: company_faq
2. "Question: ما هي شروط الاشتراك في الخدمات؟
Answer: باستخدامك لخدمات الشركة، فإنك توافق على الالتزام بالشروط والأحكام."
   Source: company_faq
3. "Question: ما هي الشروط التي تحكم استخدام خدمات الشركة؟
Answer: من الشروط الأساسية عدم استخدام الخدمة في حالات الطوارئ، وتقديم معلومات صحيحة وكاملة ودقيقة، وأن الشركة غير مسؤولة عن أي سوء استخدام للخدمة."
   Source: company_faq
4. "Question: ما هي أنواع الباقات والعضويات التي تقدمونها؟
Answer: تقدم الشركة خيارات اشتراك شهرية أو سنوية توفر خصومات إضافية ووصولاً غير محدود لبعض الخدمات."
   Source: company_faq
5. "Question: هل تقدمون خدمات للشركات؟
Answer: نعم، نوفر حزم رعاية صحية رقمية للشركات والمؤسسات لرعاية موظفيهم، وبرامج صحية مخصصة لتعزيز رفاهية الموظفين."
   Source: company_faq
6. "Question: ما هي الخدمات التي تقدمونها للمتابعة بعد العمليات الجراحية؟
Answer: نوفر جلسات متابعة مستمرة للحالات بعد العمليات الجراحية."
   Source: company_faq
7. "Question: ما هي أنواع الاستشارات التي تقدمونها للأطفال؟
Answer: نقدم استشارات مع أطباء متخصصين في تخصص الأطفال."
   Source: company_faq
8. "Question: ما هي أنواع الاستشارات التي تقدمونها للأطباء العامين؟
Answer: نقدم استشارات فورية ومجدولة مع أطباء عامين."
   Source: company_faq
9. "Question: ما هي أنواع البطاقات البنكية التي تقبلونها للدفع؟
Answer: نقبل جميع البطاقات الائتمانية والخصم المباشر الرئيسية (فيزا / ماستركارد / مدى)."
   Source: company_faq
10. "Question: ما هي برامج الطب عن بُعد التي تقدمونها للمؤسسات؟
Answer: نقدم حزم رعاية صحية رقمية للشركات لرعاية موظفيهم، وبرامج صحية مخصصة لتعزيز رفاهيتهم."
   Source: company_faq

**intfloat/multilingual-e5-small:**

*Vector Retriever (Time: 0.014s):*
1. "Question: ما هي الخدمات التي تقدمها الشركة؟
Answer: تشمل الخدمات الاستشارات الطبية عبر الفيديو، الكشف الجسدي في العيادة، مراجعة التقارير الطبية والتحاليل، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، وخدمات الرعاية النفسية والدعم الصحي. كما تقدم الشركة تحليل نتائج المختبر، نظام السجل..."
   Source: company_faq
2. "Question: كيف تضمنون جودة الرعاية الصحية؟
Answer: نضمن أن جميع خدماتنا الطبية تُقدم من قبل أطباء ومتخصصين مؤهلين تأهيلاً عاليًا ويتمتعون بخبرة واسعة في مجالاتهم."
   Source: company_faq
3. "تقديم خدمات طبية **موثوقة، آمنة، وسهلة الوصول** لجميع أفراد المجتمع، مع التركيز على الكفاءة المهنية والشفافية التامة، وذلك من خلال تسخير **أحدث التكنولوجيا الرقمية** لبناء جسر فعال بين المرضى والخبراء الطبيين. نلتزم بتقديم تجربة رعاية صحية سلسة وشاملة، تراعي احتياجات كل فرد وتضمن خصوصيته وأمن معلوما..."
   Source: company_info
4. "Question: ما هي أنواع الخدمات التي يمكن أن أحصل عليها للرعاية النفسية؟
Answer: يمكنك الحصول على جلسات استشارية مع أخصائيين نفسيين ومعالجين سلوكيين."
   Source: company_faq
5. "Question: ماذا تتضمن خدمة الرعاية النفسية؟
Answer: تتضمن جلسات استشارية مع أخصائيين نفسيين ومعالجين سلوكيين، وتقديم استشارات في مجالات الصحة العامة وإدارة الإجهاد."
   Source: company_faq

*Hybrid Retriever (Time: 0.012s):*
1. "Question: ما هي الخدمات التي تقدمها الشركة؟
Answer: تشمل الخدمات الاستشارات الطبية عبر الفيديو، الكشف الجسدي في العيادة، مراجعة التقارير الطبية والتحاليل، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، وخدمات الرعاية النفسية والدعم الصحي. كما تقدم الشركة تحليل نتائج المختبر، نظام السجل..."
   Source: company_faq
2. "Question: كيف تضمنون جودة الرعاية الصحية؟
Answer: نضمن أن جميع خدماتنا الطبية تُقدم من قبل أطباء ومتخصصين مؤهلين تأهيلاً عاليًا ويتمتعون بخبرة واسعة في مجالاتهم."
   Source: company_faq
3. "تقديم خدمات طبية **موثوقة، آمنة، وسهلة الوصول** لجميع أفراد المجتمع، مع التركيز على الكفاءة المهنية والشفافية التامة، وذلك من خلال تسخير **أحدث التكنولوجيا الرقمية** لبناء جسر فعال بين المرضى والخبراء الطبيين. نلتزم بتقديم تجربة رعاية صحية سلسة وشاملة، تراعي احتياجات كل فرد وتضمن خصوصيته وأمن معلوما..."
   Source: company_info
4. "Question: ما هي أنواع الخدمات التي يمكن أن أحصل عليها للرعاية النفسية؟
Answer: يمكنك الحصول على جلسات استشارية مع أخصائيين نفسيين ومعالجين سلوكيين."
   Source: company_faq
5. "Question: ماذا تتضمن خدمة الرعاية النفسية؟
Answer: تتضمن جلسات استشارية مع أخصائيين نفسيين ومعالجين سلوكيين، وتقديم استشارات في مجالات الصحة العامة وإدارة الإجهاد."
   Source: company_faq
6. "Question: ما هي الخدمات التي تقدمونها للمتابعة بعد العمليات الجراحية؟
Answer: نوفر جلسات متابعة مستمرة للحالات بعد العمليات الجراحية."
   Source: company_faq
7. "Question: ما هي أنواع الاستشارات التي تقدمونها للأطفال؟
Answer: نقدم استشارات مع أطباء متخصصين في تخصص الأطفال."
   Source: company_faq
8. "Question: ما هي أنواع الاستشارات التي تقدمونها للأطباء العامين؟
Answer: نقدم استشارات فورية ومجدولة مع أطباء عامين."
   Source: company_faq
9. "Question: ما هي أنواع البطاقات البنكية التي تقبلونها للدفع؟
Answer: نقبل جميع البطاقات الائتمانية والخصم المباشر الرئيسية (فيزا / ماستركارد / مدى)."
   Source: company_faq
10. "Question: ما هي برامج الطب عن بُعد التي تقدمونها للمؤسسات؟
Answer: نقدم حزم رعاية صحية رقمية للشركات لرعاية موظفيهم، وبرامج صحية مخصصة لتعزيز رفاهيتهم."
   Source: company_faq

**ibm-granite/granite-embedding-107m-multilingual:**

*Vector Retriever (Time: 0.007s):*
1. "Question: ما هي الخدمات التي تقدمها الشركة؟
Answer: تشمل الخدمات الاستشارات الطبية عبر الفيديو، الكشف الجسدي في العيادة، مراجعة التقارير الطبية والتحاليل، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، وخدمات الرعاية النفسية والدعم الصحي. كما تقدم الشركة تحليل نتائج المختبر، نظام السجل..."
   Source: company_faq
2. "Question: هل تقدمون خدمات للعائلات؟
Answer: نعم، نقدم خيارات اشتراك متعددة تناسب احتياجات الأفراد والعائلات."
   Source: company_faq
3. "تقديم خدمات طبية **موثوقة، آمنة، وسهلة الوصول** لجميع أفراد المجتمع، مع التركيز على الكفاءة المهنية والشفافية التامة، وذلك من خلال تسخير **أحدث التكنولوجيا الرقمية** لبناء جسر فعال بين المرضى والخبراء الطبيين. نلتزم بتقديم تجربة رعاية صحية سلسة وشاملة، تراعي احتياجات كل فرد وتضمن خصوصيته وأمن معلوما..."
   Source: company_info
4. "Question: ما هي وظيفة خدمة الصيدلة والتوصيل المنزلي؟
Answer: تشمل صرف الوصفات الطبية الإلكترونية وتوصيل الأدوية والمستلزمات الطبية إلى باب المنزل بسرعة وأمان."
   Source: company_faq
5. "Question: ما هي خدمات الدعم الفني المتاحة؟
Answer: يتوفر الدعم الفني عبر الدردشة المباشرة (24/7) على الموقع الإلكتروني، والهاتف خلال ساعات العمل، والبريد الإلكتروني. كما يوجد قسم للمساعدة والأسئلة الشائعة."
   Source: company_faq

*Hybrid Retriever (Time: 0.008s):*
1. "Question: ما هي الخدمات التي تقدمها الشركة؟
Answer: تشمل الخدمات الاستشارات الطبية عبر الفيديو، الكشف الجسدي في العيادة، مراجعة التقارير الطبية والتحاليل، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، وخدمات الرعاية النفسية والدعم الصحي. كما تقدم الشركة تحليل نتائج المختبر، نظام السجل..."
   Source: company_faq
2. "Question: هل تقدمون خدمات للعائلات؟
Answer: نعم، نقدم خيارات اشتراك متعددة تناسب احتياجات الأفراد والعائلات."
   Source: company_faq
3. "تقديم خدمات طبية **موثوقة، آمنة، وسهلة الوصول** لجميع أفراد المجتمع، مع التركيز على الكفاءة المهنية والشفافية التامة، وذلك من خلال تسخير **أحدث التكنولوجيا الرقمية** لبناء جسر فعال بين المرضى والخبراء الطبيين. نلتزم بتقديم تجربة رعاية صحية سلسة وشاملة، تراعي احتياجات كل فرد وتضمن خصوصيته وأمن معلوما..."
   Source: company_info
4. "Question: ما هي وظيفة خدمة الصيدلة والتوصيل المنزلي؟
Answer: تشمل صرف الوصفات الطبية الإلكترونية وتوصيل الأدوية والمستلزمات الطبية إلى باب المنزل بسرعة وأمان."
   Source: company_faq
5. "Question: ما هي خدمات الدعم الفني المتاحة؟
Answer: يتوفر الدعم الفني عبر الدردشة المباشرة (24/7) على الموقع الإلكتروني، والهاتف خلال ساعات العمل، والبريد الإلكتروني. كما يوجد قسم للمساعدة والأسئلة الشائعة."
   Source: company_faq
6. "Question: ما هي الخدمات التي تقدمونها للمتابعة بعد العمليات الجراحية؟
Answer: نوفر جلسات متابعة مستمرة للحالات بعد العمليات الجراحية."
   Source: company_faq
7. "Question: ما هي أنواع الاستشارات التي تقدمونها للأطفال؟
Answer: نقدم استشارات مع أطباء متخصصين في تخصص الأطفال."
   Source: company_faq
8. "Question: ما هي أنواع الاستشارات التي تقدمونها للأطباء العامين؟
Answer: نقدم استشارات فورية ومجدولة مع أطباء عامين."
   Source: company_faq
9. "Question: ما هي أنواع البطاقات البنكية التي تقبلونها للدفع؟
Answer: نقبل جميع البطاقات الائتمانية والخصم المباشر الرئيسية (فيزا / ماستركارد / مدى)."
   Source: company_faq
10. "Question: ما هي برامج الطب عن بُعد التي تقدمونها للمؤسسات؟
Answer: نقدم حزم رعاية صحية رقمية للشركات لرعاية موظفيهم، وبرامج صحية مخصصة لتعزيز رفاهيتهم."
   Source: company_faq

### Query 2: "كيف يمكنني التواصل معكم"

**sentence-transformers/paraphrase-multilingual-mpnet-base-v2:**

*Vector Retriever (Time: 0.027s):*
1. "Question: ما هي طرق التواصل الأخرى المتاحة غير الهاتف والبريد الإلكتروني؟
Answer: يمكن التواصل عبر الموقع الإلكتروني ووسائل التواصل الاجتماعي مثل تويتر، لينكد إن، وفيسبوك."
   Source: company_faq
2. "Question: هل يمكنني الحصول على استشارة عن طريق الهاتف؟
Answer: نعم، من بين الخدمات المتاحة التواصل الصوتي والمرئي."
   Source: company_faq
3. "Question: ما هي الإجراءات المتبعة في حال انقطاع الاتصال أثناء الاستشارة؟
Answer: في حال انقطاع الاتصال، سيحاول الطبيب أو فريق الدعم إعادة الاتصال بك لإكمال الاستشارة."
   Source: company_faq
4. "Question: كيف يمكنني التواصل مع الدعم الفني عبر الهاتف؟
Answer: يتوفر فريق دعم مخصص جاهز للإجابة على الاستفسارات التقنية والخدمية خلال ساعات العمل الرسمية."
   Source: company_faq
5. "Question: هل يمكنني تقديم ملاحظاتي أو مشاكل عبر البريد الإلكتروني؟
Answer: نعم، يمكنك إرسال استفساراتكم أو مشاكلكم عبر البريد الإلكتروني، وسيقوم فريق الدعم بالرد في أقرب وقت."
   Source: company_faq

*Hybrid Retriever (Time: 0.029s):*
1. "Question: ما هي طرق التواصل الأخرى المتاحة غير الهاتف والبريد الإلكتروني؟
Answer: يمكن التواصل عبر الموقع الإلكتروني ووسائل التواصل الاجتماعي مثل تويتر، لينكد إن، وفيسبوك."
   Source: company_faq
2. "Question: كيف يمكنني التواصل مع الدعم الفني عبر الهاتف؟
Answer: يتوفر فريق دعم مخصص جاهز للإجابة على الاستفسارات التقنية والخدمية خلال ساعات العمل الرسمية."
   Source: company_faq
3. "Question: هل يمكنني الحصول على استشارة عن طريق الهاتف؟
Answer: نعم، من بين الخدمات المتاحة التواصل الصوتي والمرئي."
   Source: company_faq
4. "Question: ما هي الإجراءات المتبعة في حال انقطاع الاتصال أثناء الاستشارة؟
Answer: في حال انقطاع الاتصال، سيحاول الطبيب أو فريق الدعم إعادة الاتصال بك لإكمال الاستشارة."
   Source: company_faq
5. "Question: هل يمكنني تقديم ملاحظاتي أو مشاكل عبر البريد الإلكتروني؟
Answer: نعم، يمكنك إرسال استفساراتكم أو مشاكلكم عبر البريد الإلكتروني، وسيقوم فريق الدعم بالرد في أقرب وقت."
   Source: company_faq
6. "Question: كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟
Answer: سيتم الإعلان عن أي تغييرات في أوقات العمل خلال العطلات مسبقًا عبر الموقع ووسائل التواصل الاجتماعي."
   Source: company_faq
7. "Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكن إلغاء الاشتراك عبر تسجيل الدخول إلى حسابك على الموقع ثم التوجه إلى "إدارة الاشتراكات". أو يمكنك التواصل مع فريق الدعم الفني للمساعدة في إتمام عملية الإلغاء."
   Source: company_faq
8. "Question: ما هي وسائل التواصل الاجتماعي الخاصة بالشركة؟
Answer: يمكن التواصل مع الشركة عبر تويتر (X)، لينكد إن، فيسبوك، وإنستغرام."
   Source: company_faq

**intfloat/multilingual-e5-small:**

*Vector Retriever (Time: 0.014s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني عبر الهاتف؟
Answer: يتوفر فريق دعم مخصص جاهز للإجابة على الاستفسارات التقنية والخدمية خلال ساعات العمل الرسمية."
   Source: company_faq
2. "Question: هل يمكنني الحصول على استشارة عن طريق الهاتف؟
Answer: نعم، من بين الخدمات المتاحة التواصل الصوتي والمرئي."
   Source: company_faq
3. "نحن هنا لخدمتكم والإجابة على استفساراتكم. يمكنكم التواصل معنا عبر القنوات التالية:  
* **الهاتف:** 9200-000-000 (متوفر خلال ساعات العمل الرسمية)
* **البريد الإلكتروني:** support@alshifa-care.com (نعد بالرد على استفساراتكم خلال 24 ساعة عمل)
* **الموقع الإلكتروني:** [www.alshifa-care.com](https://www...."
   Source: company_info
4. "Question: كيف يمكنني الاشتراك في خدماتكم؟
Answer: يمكنك الاشتراك بزيارة الموقع الإلكتروني وإنشاء حساب، ثم اختيار الباقة المناسبة، وإدخال معلومات الدفع."
   Source: company_faq
5. "Question: ماذا يحدث إذا واجهت صعوبة في إلغاء الاشتراك؟
Answer: يمكنك التواصل مع فريق الدعم الفني لمساعدتك في إتمام عملية الإلغاء."
   Source: company_faq

*Hybrid Retriever (Time: 0.013s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني عبر الهاتف؟
Answer: يتوفر فريق دعم مخصص جاهز للإجابة على الاستفسارات التقنية والخدمية خلال ساعات العمل الرسمية."
   Source: company_faq
2. "Question: هل يمكنني الحصول على استشارة عن طريق الهاتف؟
Answer: نعم، من بين الخدمات المتاحة التواصل الصوتي والمرئي."
   Source: company_faq
3. "نحن هنا لخدمتكم والإجابة على استفساراتكم. يمكنكم التواصل معنا عبر القنوات التالية:  
* **الهاتف:** 9200-000-000 (متوفر خلال ساعات العمل الرسمية)
* **البريد الإلكتروني:** support@alshifa-care.com (نعد بالرد على استفساراتكم خلال 24 ساعة عمل)
* **الموقع الإلكتروني:** [www.alshifa-care.com](https://www...."
   Source: company_info
4. "Question: كيف يمكنني الاشتراك في خدماتكم؟
Answer: يمكنك الاشتراك بزيارة الموقع الإلكتروني وإنشاء حساب، ثم اختيار الباقة المناسبة، وإدخال معلومات الدفع."
   Source: company_faq
5. "Question: ماذا يحدث إذا واجهت صعوبة في إلغاء الاشتراك؟
Answer: يمكنك التواصل مع فريق الدعم الفني لمساعدتك في إتمام عملية الإلغاء."
   Source: company_faq
6. "Question: كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟
Answer: سيتم الإعلان عن أي تغييرات في أوقات العمل خلال العطلات مسبقًا عبر الموقع ووسائل التواصل الاجتماعي."
   Source: company_faq
7. "Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكن إلغاء الاشتراك عبر تسجيل الدخول إلى حسابك على الموقع ثم التوجه إلى "إدارة الاشتراكات". أو يمكنك التواصل مع فريق الدعم الفني للمساعدة في إتمام عملية الإلغاء."
   Source: company_faq
8. "Question: ما هي طرق التواصل الأخرى المتاحة غير الهاتف والبريد الإلكتروني؟
Answer: يمكن التواصل عبر الموقع الإلكتروني ووسائل التواصل الاجتماعي مثل تويتر، لينكد إن، وفيسبوك."
   Source: company_faq
9. "Question: ما هي وسائل التواصل الاجتماعي الخاصة بالشركة؟
Answer: يمكن التواصل مع الشركة عبر تويتر (X)، لينكد إن، فيسبوك، وإنستغرام."
   Source: company_faq

**ibm-granite/granite-embedding-107m-multilingual:**

*Vector Retriever (Time: 0.006s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني عبر الهاتف؟
Answer: يتوفر فريق دعم مخصص جاهز للإجابة على الاستفسارات التقنية والخدمية خلال ساعات العمل الرسمية."
   Source: company_faq
2. "Question: ما هي طرق التواصل الأخرى المتاحة غير الهاتف والبريد الإلكتروني؟
Answer: يمكن التواصل عبر الموقع الإلكتروني ووسائل التواصل الاجتماعي مثل تويتر، لينكد إن، وفيسبوك."
   Source: company_faq
3. "Question: ما هي وسائل التواصل الاجتماعي الخاصة بالشركة؟
Answer: يمكن التواصل مع الشركة عبر تويتر (X)، لينكد إن، فيسبوك، وإنستغرام."
   Source: company_faq
4. "Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكن إلغاء الاشتراك عبر تسجيل الدخول إلى حسابك على الموقع ثم التوجه إلى "إدارة الاشتراكات". أو يمكنك التواصل مع فريق الدعم الفني للمساعدة في إتمام عملية الإلغاء."
   Source: company_faq
5. "Question: كيف يمكنني الاشتراك في خدماتكم؟
Answer: يمكنك الاشتراك بزيارة الموقع الإلكتروني وإنشاء حساب، ثم اختيار الباقة المناسبة، وإدخال معلومات الدفع."
   Source: company_faq

*Hybrid Retriever (Time: 0.007s):*
1. "Question: كيف يمكنني التواصل مع الدعم الفني عبر الهاتف؟
Answer: يتوفر فريق دعم مخصص جاهز للإجابة على الاستفسارات التقنية والخدمية خلال ساعات العمل الرسمية."
   Source: company_faq
2. "Question: ما هي طرق التواصل الأخرى المتاحة غير الهاتف والبريد الإلكتروني؟
Answer: يمكن التواصل عبر الموقع الإلكتروني ووسائل التواصل الاجتماعي مثل تويتر، لينكد إن، وفيسبوك."
   Source: company_faq
3. "Question: ما هي وسائل التواصل الاجتماعي الخاصة بالشركة؟
Answer: يمكن التواصل مع الشركة عبر تويتر (X)، لينكد إن، فيسبوك، وإنستغرام."
   Source: company_faq
4. "Question: كيف يمكنني إلغاء الاشتراك؟
Answer: يمكن إلغاء الاشتراك عبر تسجيل الدخول إلى حسابك على الموقع ثم التوجه إلى "إدارة الاشتراكات". أو يمكنك التواصل مع فريق الدعم الفني للمساعدة في إتمام عملية الإلغاء."
   Source: company_faq
5. "Question: كيف يمكنني الاشتراك في خدماتكم؟
Answer: يمكنك الاشتراك بزيارة الموقع الإلكتروني وإنشاء حساب، ثم اختيار الباقة المناسبة، وإدخال معلومات الدفع."
   Source: company_faq
6. "Question: كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟
Answer: سيتم الإعلان عن أي تغييرات في أوقات العمل خلال العطلات مسبقًا عبر الموقع ووسائل التواصل الاجتماعي."
   Source: company_faq

### Query 3: "ما هي أوقات العمل"

**sentence-transformers/paraphrase-multilingual-mpnet-base-v2:**

*Vector Retriever (Time: 0.031s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
2. "Question: ما هي أوقات العمل الرسمية للشركة؟
Answer: من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
3. "Question: كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟
Answer: سيتم الإعلان عن أي تغييرات في أوقات العمل خلال العطلات مسبقًا عبر الموقع ووسائل التواصل الاجتماعي."
   Source: company_faq
4. "Question: هل الشركة تعمل في العطلات الأسبوعية؟
Answer: لا، العطلات الأسبوعية هي الجمعة والسبت."
   Source: company_faq
5. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info

*Hybrid Retriever (Time: 0.030s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
2. "Question: ما هي أوقات العمل الرسمية للشركة؟
Answer: من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
3. "Question: كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟
Answer: سيتم الإعلان عن أي تغييرات في أوقات العمل خلال العطلات مسبقًا عبر الموقع ووسائل التواصل الاجتماعي."
   Source: company_faq
4. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
5. "Question: هل الشركة تعمل في العطلات الأسبوعية؟
Answer: لا، العطلات الأسبوعية هي الجمعة والسبت."
   Source: company_faq
6. "Question: ما هو نطاق عمل الشركة؟
Answer: نطاق العمل يشمل الرعاية الصحية عن بُعد."
   Source: company_faq

**intfloat/multilingual-e5-small:**

*Vector Retriever (Time: 0.012s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
2. "Question: ما هي أوقات العمل الرسمية للشركة؟
Answer: من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
3. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
4. "Question: كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟
Answer: سيتم الإعلان عن أي تغييرات في أوقات العمل خلال العطلات مسبقًا عبر الموقع ووسائل التواصل الاجتماعي."
   Source: company_faq
5. "Question: هل الشركة تعمل في العطلات الأسبوعية؟
Answer: لا، العطلات الأسبوعية هي الجمعة والسبت."
   Source: company_faq

*Hybrid Retriever (Time: 0.012s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
2. "Question: ما هي أوقات العمل الرسمية للشركة؟
Answer: من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
3. "Question: كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟
Answer: سيتم الإعلان عن أي تغييرات في أوقات العمل خلال العطلات مسبقًا عبر الموقع ووسائل التواصل الاجتماعي."
   Source: company_faq
4. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
5. "Question: هل الشركة تعمل في العطلات الأسبوعية؟
Answer: لا، العطلات الأسبوعية هي الجمعة والسبت."
   Source: company_faq
6. "Question: ما هو نطاق عمل الشركة؟
Answer: نطاق العمل يشمل الرعاية الصحية عن بُعد."
   Source: company_faq

**ibm-granite/granite-embedding-107m-multilingual:**

*Vector Retriever (Time: 0.007s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
2. "Question: ما هي أوقات العمل الرسمية للشركة؟
Answer: من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
3. "Question: كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟
Answer: سيتم الإعلان عن أي تغييرات في أوقات العمل خلال العطلات مسبقًا عبر الموقع ووسائل التواصل الاجتماعي."
   Source: company_faq
4. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
5. "Question: هل الشركة تعمل في العطلات الأسبوعية؟
Answer: لا، العطلات الأسبوعية هي الجمعة والسبت."
   Source: company_faq

*Hybrid Retriever (Time: 0.007s):*
1. "Question: ما هي أوقات عمل الشركة؟
Answer: أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
2. "Question: ما هي أوقات العمل الرسمية للشركة؟
Answer: من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST)."
   Source: company_faq
3. "Question: كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟
Answer: سيتم الإعلان عن أي تغييرات في أوقات العمل خلال العطلات مسبقًا عبر الموقع ووسائل التواصل الاجتماعي."
   Source: company_faq
4. "نحن ملتزمون بتوفير خدماتنا لكم بأقصى درجات المرونة، مع مراعاة أوقات الراحة اللازمة لفريقنا:  
* **أيام العمل:** من الأحد إلى الخميس.
* **الساعات:** 9 صباحًا حتى 9 مساءً بتوقيت المملكة العربية السعودية (EEST).
* **العطلات الأسبوعية:** الجمعة والسبت.
* **العطل الرسمية:** بالإضافة إلى العطلات الأسبوعية..."
   Source: company_info
5. "Question: هل الشركة تعمل في العطلات الأسبوعية؟
Answer: لا، العطلات الأسبوعية هي الجمعة والسبت."
   Source: company_faq
6. "Question: ما هو نطاق عمل الشركة؟
Answer: نطاق العمل يشمل الرعاية الصحية عن بُعد."
   Source: company_faq

## Performance Summary:


### sentence-transformers/paraphrase-multilingual-mpnet-base-v2
- **Total Processing Time:** 11.33 seconds
- **Vector Retriever Average Time:** 0.033 seconds per query
- **Hybrid Retriever Average Time:** 0.030 seconds per query

### intfloat/multilingual-e5-small
- **Total Processing Time:** 10.32 seconds
- **Vector Retriever Average Time:** 0.013 seconds per query
- **Hybrid Retriever Average Time:** 0.012 seconds per query

### ibm-granite/granite-embedding-107m-multilingual
- **Total Processing Time:** 8.29 seconds
- **Vector Retriever Average Time:** 0.007 seconds per query
- **Hybrid Retriever Average Time:** 0.007 seconds per query


## Evaluation Criteria:
1. **Performance Speed:** Faster retrieval times
2. **Document Relevance:** How well retrieved documents match the query intent
3. **Multilingual Support:** Better handling of Arabic queries
4. **Result Quality:** Accuracy and relevance of retrieved documents
5. **Consistency:** Stable performance and relevance across different queries

## What I Need From You:
1. **Rank the Embedding Models:** (1st, 2nd, 3rd) with justification based on both speed AND document relevance
2. **Compare Retriever Types:** Which is better - Vector or Hybrid? Consider both speed and relevance
3. **Document Relevance Analysis:** For each query, which model+retriever combination returned the most relevant documents?
4. **Final Recommendation:** Best combination (embedding model + retriever type) considering all factors
5. **Detailed Analysis:** Strengths and weaknesses of each combination in terms of:
   - Speed performance
   - Document relevance quality
   - Consistency across queries
6. **Production Considerations:** Recommendations for production environment usage

## Objective:
Select the best combination that provides the optimal balance between:
- Response speed
- Quality and relevance of retrieved results
- Efficiency in processing Arabic text
- Performance stability
- Document-query matching accuracy

## Additional Context:
- The system will be used for company information retrieval
- Arabic language support is crucial
- Both speed and accuracy are important
- The system should return highly relevant documents
- Consider scalability for production use

## Analysis Instructions:
Please analyze each query's results and evaluate:
1. Which retrieved documents are most relevant to each query
2. Which model consistently returns better relevant documents
3. Whether vector or hybrid retrieval provides better document relevance
4. The trade-off between speed and relevance quality

Please structure your response with clear sections for:
1. Document Relevance Analysis by Query
2. Speed Performance Analysis
3. Overall Model Ranking
4. Retriever Type Comparison
5. Final Recommendation with Justification


In [10]:

# Initialize ChatOpenAI
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("BASE_URL")
    )

    print("🤖 Sending results to GPT-4o for expert evaluation...")
    print("-" * 60)

    # Get LLM evaluation
    llm_response = llm.invoke(llm_prompt)

    print("🎯 GPT-4o Expert Evaluation:")
    print("=" * 60)
    display(Markdown(llm_response.content))

except Exception as e:
    print(f"❌ Error with LLM evaluation: {str(e)}")
    print("💡 Please check your OpenAI API key and connection")



🤖 Sending results to GPT-4o for expert evaluation...
------------------------------------------------------------


2025-08-07 21:30:59,447 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


🎯 GPT-4o Expert Evaluation:


# Analysis and Recommendations for Embedding Models and Retriever Types

## 1. Document Relevance Analysis by Query

### Query 1: "ما هي الخدمات الرئيسية التي تقدمونها"
- **Most Relevant Results:**
  - All models retrieved the most relevant document: "ما هي الخدمات التي تقدمها الشركة؟" as the top result.
  - **sentence-transformers/paraphrase-multilingual-mpnet-base-v2** and **intfloat/multilingual-e5-small** (Hybrid Retriever) provided additional relevant documents, such as "ما هي الخدمات التي تقدمونها للمتابعة بعد العمليات الجراحية؟" and "ما هي أنواع الاستشارات التي تقدمونها للأطفال؟".
  - **ibm-granite/granite-embedding-107m-multilingual** retrieved fewer relevant documents overall, with some results being tangential (e.g., "ما هي خدمات الدعم الفني المتاحة؟").

- **Best Combination:** 
  - **intfloat/multilingual-e5-small + Hybrid Retriever** provided the most comprehensive and relevant results for this query.

---

### Query 2: "كيف يمكنني التواصل معكم"
- **Most Relevant Results:**
  - All models retrieved relevant documents, such as "كيف يمكنني التواصل مع الدعم الفني عبر الهاتف؟" and "ما هي طرق التواصل الأخرى المتاحة غير الهاتف والبريد الإلكتروني؟".
  - **intfloat/multilingual-e5-small** (Hybrid Retriever) and **sentence-transformers/paraphrase-multilingual-mpnet-base-v2** (Hybrid Retriever) included additional relevant results, such as "ما هي وسائل التواصل الاجتماعي الخاصة بالشركة؟".
  - **ibm-granite/granite-embedding-107m-multilingual** retrieved fewer results and missed some key relevant documents.

- **Best Combination:** 
  - **intfloat/multilingual-e5-small + Hybrid Retriever** provided the most relevant and diverse results.

---

### Query 3: "ما هي أوقات العمل"
- **Most Relevant Results:**
  - All models retrieved the most relevant document: "ما هي أوقات عمل الشركة؟".
  - **sentence-transformers/paraphrase-multilingual-mpnet-base-v2** and **intfloat/multilingual-e5-small** (Hybrid Retriever) provided additional relevant documents, such as "كيف يمكنني معرفة التغييرات في أوقات العمل خلال العطلات؟".
  - **ibm-granite/granite-embedding-107m-multilingual** retrieved fewer relevant documents and missed some secondary relevant results.

- **Best Combination:** 
  - **sentence-transformers/paraphrase-multilingual-mpnet-base-v2 + Hybrid Retriever** provided the most comprehensive results for this query.

---

### Summary of Document Relevance:
- **intfloat/multilingual-e5-small + Hybrid Retriever** consistently provided the most relevant and diverse results across all queries.
- **sentence-transformers/paraphrase-multilingual-mpnet-base-v2 + Hybrid Retriever** was a close second, with slightly better performance for Query 3.
- **ibm-granite/granite-embedding-107m-multilingual** lagged behind in relevance, often missing secondary relevant documents.

---

## 2. Speed Performance Analysis
- **ibm-granite/granite-embedding-107m-multilingual** was the fastest model, with an average retrieval time of **0.007 seconds** for both Vector and Hybrid retrievers.
- **intfloat/multilingual-e5-small** was the second fastest, with an average retrieval time of **0.013 seconds** for Vector and **0.012 seconds** for Hybrid.
- **sentence-transformers/paraphrase-multilingual-mpnet-base-v2** was the slowest, with an average retrieval time of **0.033 seconds** for Vector and **0.030 seconds** for Hybrid.

### Trade-off Between Speed and Relevance:
- While **ibm-granite/granite-embedding-107m-multilingual** was the fastest, its document relevance was the weakest.
- **intfloat/multilingual-e5-small** offered a good balance between speed and relevance, being significantly faster than **sentence-transformers/paraphrase-multilingual-mpnet-base-v2** while maintaining high relevance.

---

## 3. Overall Model Ranking
### Based on Speed, Relevance, and Consistency:
1. **intfloat/multilingual-e5-small**:
   - Strengths: Excellent relevance, fast retrieval times, and consistent performance across queries.
   - Weaknesses: Slightly less comprehensive than **sentence-transformers/paraphrase-multilingual-mpnet-base-v2** for Query 3.
2. **sentence-transformers/paraphrase-multilingual-mpnet-base-v2**:
   - Strengths: High relevance and comprehensive results, especially for complex queries.
   - Weaknesses: Slowest retrieval times, which may impact scalability in production.
3. **ibm-granite/granite-embedding-107m-multilingual**:
   - Strengths: Fastest retrieval times.
   - Weaknesses: Weaker relevance and less comprehensive results.

---

## 4. Retriever Type Comparison
### Vector Retriever:
- Faster than Hybrid Retriever across all models.
- However, it often retrieved fewer relevant documents, especially for queries requiring nuanced understanding.

### Hybrid Retriever:
- Consistently retrieved more relevant and diverse documents across all models.
- Slightly slower than Vector Retriever, but the difference was negligible (e.g., 0.001-0.002 seconds on average).

### Recommendation:
- **Hybrid Retriever** is the better choice, as the slight trade-off in speed is outweighed by the significant improvement in document relevance.

---

## 5. Final Recommendation with Justification
### Best Combination:
- **intfloat/multilingual-e5-small + Hybrid Retriever**

### Justification:
1. **Relevance:** Consistently retrieved the most relevant and diverse documents across all queries.
2. **Speed:** Fast retrieval times (0.012 seconds on average), making it suitable for production use.
3. **Consistency:** Stable performance across different query types.
4. **Scalability:** Faster than **sentence-transformers/paraphrase-multilingual-mpnet-base-v2**, making it more scalable for high-traffic environments.
5. **Multilingual Support:** Handled Arabic queries effectively, with no noticeable degradation in relevance.

---

## 6. Detailed Analysis of Strengths and Weaknesses

### **sentence-transformers/paraphrase-multilingual-mpnet-base-v2**
- **Strengths:**
  - High-quality embeddings with strong semantic understanding.
  - Comprehensive results for complex queries.
- **Weaknesses:**
  - Slowest retrieval times, which may impact scalability.
  - Marginally less consistent than **intfloat/multilingual-e5-small**.

### **intfloat/multilingual-e5-small**
- **Strengths:**
  - Excellent balance between speed and relevance.
  - Consistently retrieved diverse and relevant documents.
  - Fast enough for production use.
- **Weaknesses:**
  - Slightly less comprehensive than **sentence-transformers/paraphrase-multilingual-mpnet-base-v2** for some queries.

### **ibm-granite/granite-embedding-107m-multilingual**
- **Strengths:**
  - Fastest retrieval times, making it highly scalable.
- **Weaknesses:**
  - Weaker relevance and less comprehensive results.
  - Missed secondary relevant documents for most queries.

---

## 7. Production Considerations
1. **Scalability:** 
   - **intfloat/multilingual-e5-small** is fast enough to handle high query volumes while maintaining relevance.
   - **Hybrid Retriever** ensures high-quality results, even for complex queries.
2. **Arabic Language Support:**
   - Both **intfloat/multilingual-e5-small** and **sentence-transformers/paraphrase-multilingual-mpnet-base-v2** handle Arabic queries effectively.
3. **Response Time:**
   - While **ibm-granite/granite-embedding-107m-multilingual** is the fastest, its weaker relevance makes it unsuitable for production.
4. **Relevance vs. Speed Trade-off:**
   - **intfloat/multilingual-e5-small + Hybrid Retriever** strikes the best balance, ensuring both speed and relevance.

---

## 8. Final Recommendation for Production
- **Embedding Model:** **intfloat/multilingual-e5-small**
- **Retriever Type:** **Hybrid Retriever**

This combination provides the optimal balance between speed, relevance, and scalability, making it the best choice for a company information retrieval system with strong Arabic language support.

### 4.4 Create Vector Store with "intfloat/multilingual-e5-small" and Save it in "vectorstore/"











In [13]:
model_name = "intfloat/multilingual-e5-small"

persist_dir = VECTOR_STORE_DIR

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

# Create vector store
vector_store = FAISS.from_documents(company_chunks, embeddings)

# Save the vector store
vector_store.save_local(persist_dir)
print("💾 Vector store saved successfully")

2025-08-07 21:33:58,782 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-small


💾 Vector store saved successfully


### 4.5 Create Hybrid Retriever

In [14]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever


model_name = "intfloat/multilingual-e5-small"

persist_dir = VECTOR_STORE_DIR

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)
print("📂 Loading vector store...")
vector_store = FAISS.load_local(persist_dir, embeddings, allow_dangerous_deserialization=True)
    
k = 5

# Create vector retriever
print("🔍 Creating vector retriever...")
vector_retriever = vector_store.as_retriever(search_kwargs={"k": k})

# Create BM25 retriever
print("📝 Creating BM25 retriever...")
bm25_retriever = BM25Retriever.from_documents(company_chunks)
bm25_retriever.k = k

# Create hybrid retriever
print("🔄 Creating hybrid retriever...")
hybrid_retriever = EnsembleRetriever(
       retrievers=[bm25_retriever, vector_retriever],
       weights=[0.2, 0.8]
   )


print("✅ Retrievers created and hybrid retriever is ready.")

2025-08-07 21:34:14,299 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-small


📂 Loading vector store...
🔍 Creating vector retriever...
📝 Creating BM25 retriever...
🔄 Creating hybrid retriever...
✅ Retrievers created and hybrid retriever is ready.


In [15]:
query = "كيف يمكنني التواصل معكم"
retrieved_docs = hybrid_retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"--- Document {i} ---")
    print("Metadata:", doc.metadata)
    print("Content:", doc.page_content)
    print()

--- Document 1 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '109'}
Content: Question: كيف يمكنني التواصل مع الدعم الفني عبر الهاتف؟
Answer: يتوفر فريق دعم مخصص جاهز للإجابة على الاستفسارات التقنية والخدمية خلال ساعات العمل الرسمية.

--- Document 2 ---
Metadata: {'source': 'company_faq', 'type': 'faq', 'doc_id': '156'}
Content: Question: هل يمكنني الحصول على استشارة عن طريق الهاتف؟
Answer: نعم، من بين الخدمات المتاحة التواصل الصوتي والمرئي.

--- Document 3 ---
Metadata: {'company_title': 'شركة الشفاء الرقمية للرعاية الصحية', 'section': '📞 معلومات الاتصال', 'source': 'company_info', 'type': 'general_info', 'filename': 'info.md', 'doc_id': 'company_info_main'}
Content: نحن هنا لخدمتكم والإجابة على استفساراتكم. يمكنكم التواصل معنا عبر القنوات التالية:  
* **الهاتف:** 9200-000-000 (متوفر خلال ساعات العمل الرسمية)
* **البريد الإلكتروني:** support@alshifa-care.com (نعد بالرد على استفساراتكم خلال 24 ساعة عمل)
* **الموقع الإلكتروني:** [www.alshifa-care.com](https://www.googl

## 5. Tools

### 5.1 Create a Hybrid Retriever Tool

In [16]:

retriever_tool = create_retriever_tool(
    hybrid_retriever,
    name="company_knowledge_tool",
    description="Useful for answering questions about the company, its history, products, and internal policies. Do not use for general or medical questions."
)

print(retriever_tool.invoke({"query": "ما هي الخدمات التي تقدمونها"}))

Question: ما هي الخدمات التي تقدمونها للمتابعة بعد العمليات الجراحية؟
Answer: نوفر جلسات متابعة مستمرة للحالات بعد العمليات الجراحية.

Question: ما هي أنواع الخدمات التي يمكن أن أحصل عليها للرعاية النفسية؟
Answer: يمكنك الحصول على جلسات استشارية مع أخصائيين نفسيين ومعالجين سلوكيين.

Question: ما هي الخدمات التي تقدمها الشركة؟
Answer: تشمل الخدمات الاستشارات الطبية عبر الفيديو، الكشف الجسدي في العيادة، مراجعة التقارير الطبية والتحاليل، خدمات الصيدلة والتوصيل المنزلي، حجز مواعيد مع أطباء متخصصين، وخدمات الرعاية النفسية والدعم الصحي. كما تقدم الشركة تحليل نتائج المختبر، نظام السجلات الطبية الإلكترونية الموحدة (EHR)، برامج الصحة الوقائية، وخدمات الطب عن بُعد للمؤسسات والشركات.

Question: هل تقدمون خدمات للشركات؟
Answer: نعم، نوفر حزم رعاية صحية رقمية للشركات والمؤسسات لرعاية موظفيهم، وبرامج صحية مخصصة لتعزيز رفاهية الموظفين.

Question: ماذا تتضمن خدمة الرعاية النفسية؟
Answer: تتضمن جلسات استشارية مع أخصائيين نفسيين ومعالجين سلوكيين، وتقديم استشارات في مجالات الصحة العامة وإدارة الإجهاد.

Q

### 5.2 Create a Web Search Tool


In [17]:

websearch_tool = TavilySearchResults(
    max_results=4,
    include_answer=True,
    include_raw_content=False,
    include_images=False,
    # search_depth="advanced",
    # include_domains = [],
    # exclude_domains = [],
    name="Tavily_Search_Tool",
    description="A search tool for answering general medical questions, current events, or any queries not related to the company's internal data. Always use this for medical advice or for up-to-date information."
)

results = websearch_tool.invoke({'query' : "ما هي اعراض جرثومة المعدة"})
for i in results:
    print(f"Title: {i['title']}")
    print(f"Content: {i['content']}")
    print(f"URL: {i['url']}")
    print("-" * 50)


Title: ما هي أعراض جرثومة المعدة بالتفصيل؟ ومضاعفاتها
Content: تتعدد أعراضجرثومة المعدة، وتشتمل على ما يلي:

اقرأ أيضاً:أعراض جرثومة المعدة النفسية

هل لديك اسئلة متعلقة في هذا الموضوع؟   
اسال سينا، ذكاء اصطناعي للاجابة عن كل اسئلتك الطبية

اكتب سؤالك هنا، سينا يجهز الاجابة لك

## أعراض جرثومة المعدة في البراز

تتمثل أعراض جرثومة المعدة في البراز بظهور دم فيه مما يسبب اغمقاق لونه، وذلك لأن جرثومة المعدة تسبب في كثير من الأحيان قرحة هضمية (بالإنجليزية: Peptic Ulcer) وهي تقرحات مفتوحة في النسيج المبطن للمعدة والإثني عشر. [...] إن الإصابة بجرثومة المعدة تزيد من فرص الإصابة بسرطان المعدة، مما يؤدي إلى ظهور أعراض قليلة في بداية حدوث الإصابة، ومن هذه الأعراض حرقة الفؤاد وبمرور الوقت، قد تلاحظ الأعراض التالية:

ونتيجة للمضاعفات الخطيرة التي يمكن أن يتسبب فيها الميكروب الحلزوني، فإنها تعتبر من الحالات التيتتطلب عناية طبية وعلاج فوري، و يتم تشخيص الإصابة بها بإجراء عدة فحوصات سريرية ومخبرية، مثل الفحص البدني، وفحوصات الدم، بالإضافة إلى تحليل جرثومة المعدة بالبراز، والتنظير الداخلي. [...] قد تظ

### 5.3 Create Current Date, Time, and Day Tool

In [18]:

@tool
def get_current_datetime_tool() -> str:
    """
    Returns the current date, time, and day of the week for Saudi Arabia (Asia/Riyadh).
    This is the only reliable source for date and time information. Use this tool
    whenever a user asks about 'today', 'now', or any other time-sensitive query.
    The output is in English but shows Saudi Arabia local time.
    """
    try:
        # Define the timezone for Saudi Arabia
        saudi_tz = pytz.timezone('Asia/Riyadh')
        
        # Get the current time in that timezone
        now_saudi = datetime.now(saudi_tz)
        
        # Manual mapping to ensure English output regardless of system locale
        days_en = {
            0: "Monday", 1: "Tuesday", 2: "Wednesday", 3: "Thursday",
            4: "Friday", 5: "Saturday", 6: "Sunday"
        }
        months_en = {
            1: "January", 2: "February", 3: "March", 4: "April",
            5: "May", 6: "June", 7: "July", 8: "August",
            9: "September", 10: "October", 11: "November", 12: "December"
        }
        
        # Get English names using manual mapping
        day_name = days_en[now_saudi.weekday()]
        month_name = months_en[now_saudi.month]
        day = now_saudi.day
        year = now_saudi.year
        
        # Format time manually to avoid locale issues
        hour = now_saudi.hour
        minute = now_saudi.minute
        
        # Convert to 12-hour format
        if hour == 0:
            hour_12 = 12
            period = "AM"
        elif hour < 12:
            hour_12 = hour
            period = "AM"
        elif hour == 12:
            hour_12 = 12
            period = "PM"
        else:
            hour_12 = hour - 12
            period = "PM"
        
        time_str = f"{hour_12:02d}:{minute:02d} {period}"
        
        # Create the final string
        return f"Current date and time in Saudi Arabia: {day_name}, {month_name} {day}, {year} at {time_str}"
    
    except Exception as e:
        return f"Error getting current datetime: {str(e)}"

# Test the tool 
current_time_info = get_current_datetime_tool.invoke({})
print(current_time_info)

Current date and time in Saudi Arabia: Thursday, August 7, 2025 at 09:34 PM


### 5.4 Create Book Consultation Tool

In [19]:
# --- Pydantic Model for Structured Tool Input ---
# All fields are now Optional to prevent validation errors at the agent level.
# The validation logic is moved inside the tool itself.
class BookingInput(BaseModel):
    """Inputs for the book_consultation tool."""
    patient_name: Optional[str] = Field(None, description="The user's full name.")
    age: Optional[str] = Field(None, description="The user's age.")
    gender: Optional[str] = Field(None, description="The user's gender.")
    contact_number: Optional[str] = Field(None, description="A phone number for confirmation.")
    email: Optional[str] = Field(None, description="An email address for confirmation and reminders.")
    reason_for_consultation: Optional[str] = Field(None, description="A brief description of the user's symptoms or reason for the visit.")
    preferred_date: Optional[str] = Field(None, description="The user's desired date for the appointment.")
    preferred_time: Optional[str] = Field(None, description="The user's desired time for the appointment.")
    specialty: Optional[str] = Field(None, description="The specific medical field needed.")
    doctor_preference: Optional[str] = Field(None, description="The name of a specific doctor, if requested.")
    consultation_type: Optional[str] = Field(None, description="The method of consultation.")

@tool(args_schema=BookingInput)
def book_consultation_tool(patient_name: Optional[str] = None, age: Optional[str] = None, gender: Optional[str] = None, contact_number: Optional[str] = None, email: Optional[str] = None, reason_for_consultation: Optional[str] = None, preferred_date: Optional[str] = None, preferred_time: Optional[str] = None, consultation_type: Optional[str] = None, specialty: Optional[str] = None, doctor_preference: Optional[str] = None) -> str:
    """
    Books a medical consultation. If required information is missing, it returns a message
    asking for the missing details. Otherwise, it returns the booking data as a JSON object.
    Email is optional and will be set to 'unknown@alshifa-care.com' if not provided.
    """
    # --- 1. Internal Validation: Check for missing required information ---
    missing_fields = []
    if not patient_name:
        missing_fields.append("اسم المريض الكامل (patient_name)")
    if not age:
        missing_fields.append("العمر (age)")
    if not gender:
        missing_fields.append("الجنس (gender)")
    if not contact_number:
        missing_fields.append("رقم التواصل (contact_number)")
    if not reason_for_consultation:
        missing_fields.append("سبب الاستشارة (reason_for_consultation)")
    if not preferred_date:
        missing_fields.append("التاريخ المفضل (preferred_date)")
    if not preferred_time:
        missing_fields.append("الوقت المفضل (preferred_time)")
    # Set default email if not provided
    if not email:
        email = "unknown@alshifa-care.com"
    # --- 2. Handle missing required fields ---
    if missing_fields:
        missing_fields_str = "\n- ".join([""] + missing_fields)  # Add newline and bullet points
        return f"""
        عذراً، لا يمكن إتمام الحجز بسبب نقص المعلومات المطلوبة. يرجى توفير:
        {missing_fields_str}
       
        Sorry, we cannot complete the booking due to missing required information. Please provide:
        {missing_fields_str}
        """
    # --- 2. If all data is present, proceed with booking ---
    booking_id = f"BK-{uuid.uuid4().hex[:8].upper()}"
    timestamp = datetime.now().isoformat()
    booking_data = {
        "booking_id": booking_id,
        "booking_timestamp": timestamp,
        "patient_details": {"name": patient_name, "age": age, "gender": gender, "contact_number": contact_number, "email": email},
        "consultation_details": {
            "reason": reason_for_consultation,
            "preferred_date": preferred_date,
            "preferred_time": preferred_time,
            "specialty": specialty or "Not Specified",
            "doctor_preference": doctor_preference or "Any",
            "type": consultation_type or "Video Call"
        }
    }
    # 3. Return the final data as a formatted JSON string
    return json.dumps(booking_data, indent=4, ensure_ascii=False)

## 6. AI Agent

In [ ]:

# Assuming retriever_tool and websearch_tool are defined correctly in a previous cell
tools = [retriever_tool, websearch_tool, get_current_datetime_tool, book_consultation_tool]


# System message template for the agent
SYSTEM_MESSAGE = """
You are an advanced medical chatbot for "Al Shifa Digital Healthcare" (شركة الشفاء الرقمية للرعاية الصحية). 
Your name is "Al Shifa Digital Assistant" (روبوت الشفاء الرقمي).

**LANGUAGE RULE: Always respond in the SAME language as the user's question (Arabic/English).**

**CORE MISSION:** 
Provide accurate, evidence-based medical information and assist with appointment booking while prioritizing patient safety.

**WORKING HOURS:**
أيام العمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً.
Working days: Sunday to Thursday, 9 AM to 9 PM.

**AVAILABLE TOOLS:**
1. **book_consultation:** For appointment booking (collect all required info first)
2. **company_knowledge_tool:** For Al Shifa services, hours, company info
3. **get_current_datetime:** For current date/time (only reliable source)
4. **Tavily_Search_Tool:** For medical questions (always use for medical queries) 

**WORKFLOW:**

**Medical Questions:**
- Start with safety disclaimer
- Use Tavily_Search_Tool for current, evidence-based information
- Never diagnose - recommend professional consultation
- **EMERGENCIES:** Immediately instruct to call 997 (emergency services) and seek immediate medical attention

**Appointment Booking:**
- **MANDATORY VALIDATION:** Before booking any appointment, MUST verify:
  - Date is Sunday through Thursday (not Friday or Saturday)
  - Time is between 9:00 AM and 9:00 PM
  - If user requests invalid day/time, inform them of working hours and ask for alternative
- Collect: patient_name, age, gender, contact_number, email, reason_for_consultation, preferred_date, preferred_time
- Use get_current_datetime for "today" requests, then check company hours
- **Validation Messages:**
  - Arabic: "عذراً، نحن نعمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً. يرجى اختيار موعد آخر."
  - English: "Sorry, we work Sunday to Thursday, 9 AM to 9 PM. Please choose another time."

**Company Questions:** 
Use company_knowledge_tool directly

**CRITICAL RULES:**
- **Safety First:** Medical emergencies → direct to call 997 immediately
- **No Diagnosis:** Provide information only, not medical diagnoses
- **Evidence-Based:** Always use Tavily_Search_Tool for medical information
- **Language Match:** Respond in user's language
- **Professional Boundaries:** Clearly state limitations when uncertain
- **Working Hours Enforcement:** Never book appointments outside working days/hours

**SAFETY DISCLAIMERS:**
- Arabic: "تنويه هام: للطوارئ اتصل بـ 997 فوراً. هذه معلومات تعليمية ولا تغني عن استشارة طبيب."
- English: "Important: For emergencies call 997 immediately. This is educational information, not medical advice."

**EMERGENCY PROTOCOL:** 
If user describes emergency symptoms (chest pain, difficulty breathing, severe bleeding, loss of consciousness, etc.), 
immediately respond:
"هذه حالة طوارئ! اتصل بـ 997 فوراً واطلب المساعدة الطبية العاجلة" / 
"This is an emergency! Call 997 immediately and seek urgent medical help."

**Language:**

- Respond in the same language as the user's query.
- الرد بنفس لغة استعلام المستخدم.

"""

# The full prompt template for the agent
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_MESSAGE),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

llm = ChatOpenAI(
    model_name="gpt-4o", 
    temperature=0,
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
)


agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template,
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

# --- Memory Setup ---
# 1. Instantiate the memory. `memory_key` must match the placeholder in the prompt.
#    `return_messages=True` ensures it returns message objects.
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=256, 
    memory_key="chat_history",
    return_messages=True
)


# 2. Create a function to run the agent with memory
def run_agent(user_input):
    # Load the conversation history from memory
    chat_history = memory.load_memory_variables({})["chat_history"]
    
    # Invoke the agent with the input and history
    response = agent_executor.invoke({
        "input": user_input,
        "chat_history": chat_history
    })
    
    # Save the context (input and output) to memory
    memory.save_context(
        {"input": user_input},
        {"output": response["output"]}
    )
    
    return response["output"]


### 6.1 Agent Execution

In [31]:
user_question_1 = "ما هي الخدمات التي تقدمونها"
response_1 = run_agent(user_question_1)
print("==="*50)
print(f"User: {user_question_1}")
print(f"Agent:")
print_markdown(response_1)



> Entering new AgentExecutor chain...


2025-08-07 21:41:23,418 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"



Invoking: `company_knowledge_tool` with `{'query': 'ما هي الخدمات التي تقدمها شركة الشفاء الرقمية للرعاية الصحية؟'}`


Question: ما هي شركة الشفاء الرقمية؟
Answer: هي شركة رائدة ومتطورة في مجال الرعاية الصحية عن بُعد (Telehealth)، تأسست لتوفير رعاية طبية شاملة وعالية الجودة عبر الإنترنت.

شركة الشفاء الرقمية هي شركة رائدة ومتطورة في مجال **الرعاية الصحية عن بُعد (Telehealth)**، تأسست على مبدأ توفير رعاية طبية شاملة وعالية الجودة عبر الإنترنت. نهدف إلى سد الفجوة بين المرضى والأطباء المتخصصين، وتمكين الأفراد من الحصول على استشارات طبية موثوقة، متابعة مستمرة، وخدمات صحية متكاملة في أي وقت ومن أي مكان، وذلك بفضل أحدث التقنيات الرقمية والكوادر الطبية المتميزة. نحن نؤمن بأن الرعاية الصحية يجب أن تكون في متناول الجميع، ولذلك نعمل بلا كلل لتقديم حلول مبتكرة تسهل وصول المرضى للخدمات التي يحتاجونها دون عناء التنقل أو الانتظار.  
---

Question: ما هو الدور الأساسي لشركة الشفاء الرقمية؟
Answer: دورها هو أن تكون شركة رائدة ومتطورة في مجال الرعاية الصحية عن بُعد.

Question: ما هو مبدأ تأسيس شركة ال

2025-08-07 21:41:24,570 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


شركة الشفاء الرقمية للرعاية الصحية تقدم مجموعة واسعة من الخدمات، منها:

- **الاستشارات الطبية عبر الفيديو** مع أطباء متخصصين.
- **الكشف الجسدي في العيادة**.
- **مراجعة التقارير الطبية والتحاليل**.
- **خدمات الصيدلة والتوصيل المنزلي**.
- **حجز مواعيد مع أطباء متخصصين**.
- **خدمات الرعاية النفسية والدعم الصحي**، بما في ذلك جلسات استشارية مع أخصائيين نفسيين ومعالجين سلوكيين.
- **تحليل نتائج المختبر**.
- **نظام السجلات الطبية الإلكترونية الموحدة (EHR)**.
- **برامج الصحة الوقائية**.
- **خدمات الطب عن بُعد للمؤسسات والشركات**.

يرجى ملاحظة أن خدمات الشركة مخصصة للحالات غير الطارئة والاستشارات الروتينية فقط. في حالات الطوارئ، يجب الاتصال برقم الطوارئ 997 أو التوجه إلى أقرب مستشفى.

> Finished chain.
User: ما هي الخدمات التي تقدمونها
Agent:


شركة الشفاء الرقمية للرعاية الصحية تقدم مجموعة واسعة من الخدمات، منها:

- **الاستشارات الطبية عبر الفيديو** مع أطباء متخصصين.
- **الكشف الجسدي في العيادة**.
- **مراجعة التقارير الطبية والتحاليل**.
- **خدمات الصيدلة والتوصيل المنزلي**.
- **حجز مواعيد مع أطباء متخصصين**.
- **خدمات الرعاية النفسية والدعم الصحي**، بما في ذلك جلسات استشارية مع أخصائيين نفسيين ومعالجين سلوكيين.
- **تحليل نتائج المختبر**.
- **نظام السجلات الطبية الإلكترونية الموحدة (EHR)**.
- **برامج الصحة الوقائية**.
- **خدمات الطب عن بُعد للمؤسسات والشركات**.

يرجى ملاحظة أن خدمات الشركة مخصصة للحالات غير الطارئة والاستشارات الروتينية فقط. في حالات الطوارئ، يجب الاتصال برقم الطوارئ 997 أو التوجه إلى أقرب مستشفى.

In [32]:
user_question_2 = "هل يمكنني حجز استشارة اليوم"
response_2 = run_agent(user_question_2)
print("==="*50)
print(f"User: {user_question_2}")
print(f"Agent:")
print_markdown(response_2)



> Entering new AgentExecutor chain...


2025-08-07 21:41:29,795 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"



Invoking: `get_current_datetime_tool` with `{}`


Current date and time in Saudi Arabia: Thursday, August 7, 2025 at 09:41 PM

2025-08-07 21:41:30,307 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


عذرًا، لا يمكن حجز استشارة اليوم لأن الوقت الحالي هو خارج ساعات العمل (9 صباحًا إلى 9 مساءً). نحن نعمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً.

يرجى اختيار موعد آخر يناسبك خلال ساعات العمل.

> Finished chain.


2025-08-07 21:41:32,671 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


User: هل يمكنني حجز استشارة اليوم
Agent:


عذرًا، لا يمكن حجز استشارة اليوم لأن الوقت الحالي هو خارج ساعات العمل (9 صباحًا إلى 9 مساءً). نحن نعمل من الأحد إلى الخميس، من 9 صباحًا حتى 9 مساءً.

يرجى اختيار موعد آخر يناسبك خلال ساعات العمل.

In [33]:
user_question_3 = "الاحد القادم"
response_3 = run_agent(user_question_3)
print("==="*50)
print(f"User: {user_question_3}")
print(f"Agent:")
print_markdown(response_3)



> Entering new AgentExecutor chain...


2025-08-07 21:41:59,913 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


لإتمام حجز الاستشارة يوم الأحد القادم، سأحتاج إلى بعض المعلومات منك:

1. **الاسم الكامل**:
2. **العمر**:
3. **الجنس**:
4. **رقم الجوال**:
5. **البريد الإلكتروني** (اختياري):
6. **سبب الاستشارة**:
7. **التخصص المطلوب**:
8. **الوقت المفضل** (بين 9 صباحًا و9 مساءً):

يرجى تزويدي بهذه التفاصيل لإتمام الحجز.

> Finished chain.
User: الاحد القادم
Agent:


لإتمام حجز الاستشارة يوم الأحد القادم، سأحتاج إلى بعض المعلومات منك:

1. **الاسم الكامل**:
2. **العمر**:
3. **الجنس**:
4. **رقم الجوال**:
5. **البريد الإلكتروني** (اختياري):
6. **سبب الاستشارة**:
7. **التخصص المطلوب**:
8. **الوقت المفضل** (بين 9 صباحًا و9 مساءً):

يرجى تزويدي بهذه التفاصيل لإتمام الحجز.

In [34]:
user_question_3 = """
أحمد محمد عبد الله  
22
ذكر
01012345678  
ahmed.abdallah@example.com  
آلام مستمرة في المعدة منذ أسبوع  
باطنه
الساعة 6 مساءً
"""
response_3 = run_agent(user_question_3)
print("==="*50)
print(f"User: {user_question_3}")
print(f"Agent:")
print_markdown(response_3)



> Entering new AgentExecutor chain...


2025-08-07 21:42:37,324 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"



Invoking: `get_current_datetime_tool` with `{}`


Current date and time in Saudi Arabia: Thursday, August 7, 2025 at 09:42 PM

2025-08-07 21:42:37,924 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"



Invoking: `book_consultation_tool` with `{'patient_name': 'أحمد محمد عبد الله', 'age': '22', 'gender': 'ذكر', 'contact_number': '01012345678', 'email': 'ahmed.abdallah@example.com', 'reason_for_consultation': 'آلام مستمرة في المعدة منذ أسبوع', 'preferred_date': '2025-08-10', 'preferred_time': '18:00', 'specialty': 'باطنه'}`


{
    "booking_id": "BK-6ADD734A",
    "booking_timestamp": "2025-08-07T21:42:38.732907",
    "patient_details": {
        "name": "أحمد محمد عبد الله",
        "age": "22",
        "gender": "ذكر",
        "contact_number": "01012345678",
        "email": "ahmed.abdallah@example.com"
    },
    "consultation_details": {
        "reason": "آلام مستمرة في المعدة منذ أسبوع",
        "preferred_date": "2025-08-10",
        "preferred_time": "18:00",
        "specialty": "باطنه",
        "doctor_preference": "Any",
        "type": "Video Call"
    }
}

2025-08-07 21:42:39,586 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


تم حجز استشارتك بنجاح!

**تفاصيل الحجز:**
- **الاسم:** أحمد محمد عبد الله  
- **العمر:** 22  
- **الجنس:** ذكر  
- **رقم الجوال:** 01012345678  
- **البريد الإلكتروني:** ahmed.abdallah@example.com  
- **سبب الاستشارة:** آلام مستمرة في المعدة منذ أسبوع  
- **التخصص:** باطنه  
- **التاريخ:** الأحد، 10 أغسطس 2025  
- **الوقت:** الساعة 6 مساءً  
- **نوع الاستشارة:** مكالمة فيديو  

**رقم الحجز:** BK-6ADD734A

إذا كنت بحاجة إلى أي مساعدة إضافية، لا تتردد في التواصل معنا. نتمنى لك الشفاء العاجل!

> Finished chain.


2025-08-07 21:42:43,829 - httpx - INFO - HTTP Request: POST https://models.inference.ai.azure.com/chat/completions "HTTP/1.1 200 OK"


User: 
أحمد محمد عبد الله  
22
ذكر
01012345678  
ahmed.abdallah@example.com  
آلام مستمرة في المعدة منذ أسبوع  
باطنه
الساعة 6 مساءً

Agent:


تم حجز استشارتك بنجاح!

**تفاصيل الحجز:**
- **الاسم:** أحمد محمد عبد الله  
- **العمر:** 22  
- **الجنس:** ذكر  
- **رقم الجوال:** 01012345678  
- **البريد الإلكتروني:** ahmed.abdallah@example.com  
- **سبب الاستشارة:** آلام مستمرة في المعدة منذ أسبوع  
- **التخصص:** باطنه  
- **التاريخ:** الأحد، 10 أغسطس 2025  
- **الوقت:** الساعة 6 مساءً  
- **نوع الاستشارة:** مكالمة فيديو  

**رقم الحجز:** BK-6ADD734A

إذا كنت بحاجة إلى أي مساعدة إضافية، لا تتردد في التواصل معنا. نتمنى لك الشفاء العاجل!

### 6.2 Display Chat History

In [35]:
chat_history = memory.load_memory_variables({})["chat_history"]
for message in chat_history:
    print_markdown(message.content)

The human asks about the services provided. The AI explains that the Digital Shifa Healthcare Company offers a wide range of services, including video consultations with specialists, in-clinic physical exams, medical report reviews, pharmacy and home delivery services, appointment booking with specialists, mental health support, lab result analysis, unified electronic health records (EHR), preventive health programs, and telemedicine services for organizations. The company focuses on non-emergency and routine consultations, advising emergency cases to contact 997 or visit the nearest hospital. The human inquires about booking a consultation today, but the AI informs them that consultations cannot be booked outside working hours (9 AM to 9 PM, Sunday to Thursday). The human requests a booking for the upcoming Sunday, and the AI asks for specific details, including the full name, age, gender, phone number, optional email, reason for consultation, required specialty, and preferred time, to complete the booking.


أحمد محمد عبد الله  
22
ذكر
01012345678  
ahmed.abdallah@example.com  
آلام مستمرة في المعدة منذ أسبوع  
باطنه
الساعة 6 مساءً


تم حجز استشارتك بنجاح!

**تفاصيل الحجز:**
- **الاسم:** أحمد محمد عبد الله  
- **العمر:** 22  
- **الجنس:** ذكر  
- **رقم الجوال:** 01012345678  
- **البريد الإلكتروني:** ahmed.abdallah@example.com  
- **سبب الاستشارة:** آلام مستمرة في المعدة منذ أسبوع  
- **التخصص:** باطنه  
- **التاريخ:** الأحد، 10 أغسطس 2025  
- **الوقت:** الساعة 6 مساءً  
- **نوع الاستشارة:** مكالمة فيديو  

**رقم الحجز:** BK-6ADD734A

إذا كنت بحاجة إلى أي مساعدة إضافية، لا تتردد في التواصل معنا. نتمنى لك الشفاء العاجل!